In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

from shapely.geometry import Point

## Filter with the Manhattan boundary

Please do not start this part if you have the `building_manhattan.geojson` file

In [2]:
# import dataset (from https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh)
gdf_building = gpd.read_file('../../data/raw/building/Building Footprints.geojson', dtype={'mpluto_bbl':str})

In [3]:
# filtering buildings with the Manhattan boundary
gdf_manhattan = gpd.read_file('../../data/raw/boundary/Borough Boundaries.geojson')
gdf_manhattan = gdf_manhattan.loc[gdf_manhattan.loc[:,'boro_name'] == 'Manhattan']

gdf_building_manhattan = gpd.sjoin(gdf_building, gdf_manhattan, how='inner', op='intersects')
gdf_building_manhattan = gdf_building_manhattan.drop(['shape_len','index_right', 'boro_code', 'boro_name', 'shape_area_right','shape_leng'], axis=1)

c:\Users\seung\anaconda3\envs\teamgigo\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


## Filter by Land use

In [4]:
gdf_pluto_manhattan = gpd.read_file('../../data/raw/landuse/pluto_manhattan.geojson', dtype={'BBL':str})

# filtering lots that has total commercial area more than 0
gdf_pluto_manhattan_commercial = gdf_pluto_manhattan.loc[gdf_pluto_manhattan.loc[:,'ComArea']>0]
gdf_pluto_manhattan_commercial.loc[:,'BBL'] = gdf_pluto_manhattan_commercial.loc[:,'BBL'].astype('int64').astype(str)

C:\Users\seung\AppData\Local\Temp\ipykernel_10916\631349462.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1000010010' '1000010201' '1000020001' ... '3022940001' '3025380001'
 '3025560041']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf_pluto_manhattan_commercial.loc[:,'BBL'] = gdf_pluto_manhattan_commercial.loc[:,'BBL'].astype('int64').astype(str)


In [5]:
# merge the building footprint and mappluto data. Office, residential and retail area will be included in the building footprint
gdf_building_manhattan_commercial = gdf_building_manhattan.merge(pd.DataFrame(gdf_pluto_manhattan_commercial.loc[:,['BBL',
                                                                                                                    'OfficeArea',
                                                                                                                    'RetailArea',
                                                                                                                    'ResArea']]), left_on='mpluto_bbl', right_on='BBL', how='inner').drop('BBL', axis=1)

In [6]:
gdf_building_manhattan_commercial = gdf_building_manhattan_commercial.drop_duplicates(subset=['bin','globalid'])

In [7]:
gdf_building_manhattan_commercial

,name,base_bbl,shape_area_left,heightroof,mpluto_bbl,cnstrct_yr,globalid,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,geometry,OfficeArea,RetailArea,ResArea
0,None,1021210037,0.0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},Constructed,2100,154,Photogramm,1062896,2017-08-22,708881,"MULTIPOLYGON (((-73.93870 40.83782, -73.93863 ...",0,2250,10344
1,None,1008870001,0.0,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},Constructed,2100,37,Photogramm,1018457,2017-08-22,584238,"MULTIPOLYGON (((-73.98237 40.74524, -73.98220 ...",50018,4000,0
2,None,1013490035,0.0,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},Constructed,2100,52,Photogramm,1039988,2017-08-22,66240,"MULTIPOLYGON (((-73.96425 40.75860, -73.96421 ...",2890,0,70110
3,None,1010610006,0.0,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},Constructed,2100,40,Photogramm,1026714,2017-08-22,23730,"MULTIPOLYGON (((-73.99058 40.76523, -73.99065 ...",0,0,0
5,None,1003060013,0.0,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},Constructed,2100,39,Photogramm,1003956,2017-08-22,512541,"MULTIPOLYGON (((-73.99304 40.71761, -73.99301 ...",0,3800,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22791,None,1010500029,0.0,45.0,1010500029,1890,{6C332B4E-94B7-47F8-AD62-325B2AAAFEC9},Merged,2100,33,Other (Man,1083736,2024-02-27,128400,"MULTIPOLYGON (((-73.99366 40.75751, -73.99354 ...",0,9411,32984
22792,None,1003060022,0.0,26.00608505,1003060022,2012,{BCEF4A7B-9A3B-45DE-BEBD-64809EAA3861},Constructed,2100,47,Other (Man,1077601,2024-02-28,1257101,"MULTIPOLYGON (((-73.99256 40.71738, -73.99256 ...",5000,5000,0
22794,None,1000920030,0.0,338.0,1000920030,2023,{5CDE3269-D9A3-4732-BBB0-AD92B46BDDF2},Constructed,2100,22,Other (Man,1091002,2024-03-01,1298488,"MULTIPOLYGON (((-74.00623 40.71090, -74.00648 ...",0,0,0
22795,None,1000920032,0.0,84.72,1000920032,1925,{5D66351B-DDFF-43FB-A05D-E5A3E861CE1B},Constructed,2100,34,Photogramm,1001278,2024-03-01,225204,"MULTIPOLYGON (((-74.00613 40.71079, -74.00637 ...",23039,2000,0


In [8]:
gdf_building_manhattan_commercial = gdf_building_manhattan_commercial.drop(['name', 'base_bbl', 'shape_area_left','lststatype', 'feat_code', 'groundelev','lstmoddate', 'doitt_id','geomsource'], axis=1)

In [9]:
gdf_building_manhattan_commercial.to_file('../../data/processed/building/building_manhattan_commercial.geojson', driver='GeoJSON')